<a href="https://colab.research.google.com/github/Agil0509/NLP/blob/main/Introduction_NLP_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-02-02 19:28:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.27, 172.217.0.91, 172.217.7.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2025-02-02 19:28:55 (55.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
!unzip '/content/nlp_getting_started.zip'

Archive:  /content/nlp_getting_started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [5]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
test_df_shuffled = test_df.sample(frac = 1, random_state= 42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
train_df_shuffled['target'].value_counts()

,count
target,
0,4342
1,3271


In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
import random

random_index = random.randint(0, len(train_df)-5)

for row in train_df[['text','target']][random_index:random_index+5].itertuples():
    _,text,target = row
    print()
    print('Target:', target, '(Real diasster)' if target >0 else '(No disaster)')
    print('Text:\n', text)
    print('-------------\n')



Target: 0 (No disaster)
Text:
 Beyonce Is my pick for http://t.co/nnMQlz91o9 Fan Army #Beyhive http://t.co/o91f3cYy0R 78
-------------


Target: 0 (No disaster)
Text:
 One Direction Is my pick for http://t.co/q2eBlOKeVE Fan Army #Directioners http://t.co/eNCmhz6y34 x1386
-------------


Target: 0 (No disaster)
Text:
 You da One 

#MTVSummerStar #VideoVeranoMTV  #MTVHottest Britney Spears Lana Del Rey
-------------


Target: 0 (No disaster)
Text:
 One Direction Is my pick for http://t.co/y9WvqKGbBI Fan Army #Directioners http://t.co/S5F9FcOmp8
-------------


Target: 0 (No disaster)
Text:
 Stony Jackson is America's last hope as he leads an army of felons thus and army rejects against the army o Satan - http://t.co/0wbEcdMHQo
-------------



In [9]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [10]:
len(train_sentences), len(val_sentences)

(6851, 762)

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

#Text Vectorization

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = 'lower_and_strip_punctuation',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 1000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = 'int',
                                   output_sequence_length = max_length)

In [15]:
text_vectorizer.adapt(train_sentences)

In [16]:
words_in_vocab = text_vectorizer.get_vocabulary()
print('vocab size:',len(words_in_vocab))
print('most used words:',words_in_vocab[:5])
print('least used words:', words_in_vocab[-5:])

vocab size: 1000
most used words: ['', '[UNK]', 'the', 'a', 'in']
least used words: ['reported', 'r', 'pray', 'playlist', 'patience']


In [17]:
sample_sentence = 'there is flood in our street'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4, 103, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print('Original version:\n',random_sentence)
print('Vectorized version')
text_vectorizer([random_sentence])

Original version:
 @cityofhummus @ILNewsFlash do you want to hear more? Displaced my parents until both died in the diasporas !
Vectorized version


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   1,  68,  12, 138,   5, 684,  51, 463,  13,   1, 293, 608,
        579,   4]])>

#Embedding

In [19]:
embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                    #   input_length = max_length
                                      mask_zero = False
                                      )
embedding

<Embedding name=embedding, built=False>

In [20]:
random_sentence = random.choice(train_sentences)
print(f'Original sentence: \n{random_sentence}')
print(f'Embedding version: \n')

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentence: 
@tinybaby @adultblackmale @mattytalks meltdown mwednesday
Embedding version: 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0242961 ,  0.0095809 , -0.02347494, ..., -0.02050045,
         -0.00730705,  0.0282908 ],
        [-0.0242961 ,  0.0095809 , -0.02347494, ..., -0.02050045,
         -0.00730705,  0.0282908 ],
        [-0.0242961 ,  0.0095809 , -0.02347494, ..., -0.02050045,
         -0.00730705,  0.0282908 ],
        ...,
        [-0.02128654, -0.04900467,  0.01726544, ...,  0.0334799 ,
          0.041879  ,  0.00555562],
        [-0.02128654, -0.04900467,  0.01726544, ...,  0.0334799 ,
          0.041879  ,  0.00555562],
        [-0.02128654, -0.04900467,  0.01726544, ...,  0.0334799 ,
          0.041879  ,  0.00555562]]], dtype=float32)>

#Model 0 - Baseline with Naive Bayes

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
model_0.score(train_sentences,train_labels)

0.887169756239965

In [23]:
baseline_score = model_0.score(val_sentences,val_labels)
print(f'Our baseline model score is {100*baseline_score:.2f}%')

Our baseline model score is 79.27%


In [24]:
# Evaluation function

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,precision_recall_fscore_support

def calculate_result(y_true, y_pred):

    model_accuracy = accuracy_score(y_true,y_pred)

    model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred, average='weighted')

    model_result = {'accuracy': model_accuracy,
                    'precision' : model_precision,
                    'recall': model_recall,
                    'f1': model_f1}
    return model_result

In [25]:
y_pred = model_0.predict(val_sentences)

baseline_results = calculate_result(val_labels, y_pred)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [26]:
precision_score(val_labels,y_pred)

0.8861788617886179

#Model 1 - A simple dense model

In [27]:
# Build model with Functional API

inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(x)

model_1 = tf.keras.models.Model(inputs = inputs, outputs = outputs)
model_1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 128,129 (500.50 KB)

 Trainable params: 128,129 (500.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('my_checkpoints.weights.h5', save_weights_only = True)

model_1_history = model_1.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = [val_sentences, val_labels],
                              callbacks = [checkpoint_cb])

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6233 - loss: 0.6503 - val_accuracy: 0.7218 - val_loss: 0.5625
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7665 - loss: 0.5072 - val_accuracy: 0.7730 - val_loss: 0.5013
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8023 - loss: 0.4468 - val_accuracy: 0.7677 - val_loss: 0.4934
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8111 - loss: 0.4123 - val_accuracy: 0.7822 - val_loss: 0.4858
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8247 - loss: 0.3950 - val_accuracy: 0.7756 - val_loss: 0.4858


In [29]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7722 - loss: 0.5074 


[0.48583677411079407, 0.7755905389785767]

In [30]:
model_1_preds = model_1.predict(val_sentences)
model_1_preds[:5],val_labels[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(array([[0.5331314 ],
        [0.7032756 ],
        [0.9710554 ],
        [0.13588199],
        [0.18131602]], dtype=float32),
 array([0, 0, 1, 1, 1]))

In [31]:
model_1_preds.shape, val_labels.shape

((762, 1), (762,))

In [32]:
model_1_preds = tf.squeeze(tf.round(model_1_preds))
model_1_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 0., 0.], dtype=float32)>

In [33]:
model_1_preds.shape

TensorShape([762])

In [34]:
model_1_results = calculate_result(val_labels, model_1_preds)
model_1_results

{'accuracy': 0.7755905511811023,
 'precision': 0.7786932114612464,
 'recall': 0.7755905511811023,
 'f1': 0.77295196996883}

In [35]:
# learned embeddings

sample_embed_learned = embedding(text_vectorizer([random_sentence]))
sample_embed_learned

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03684749, -0.00375049, -0.03602223, ..., -0.03341193,
         -0.02006156,  0.01481214],
        [-0.03684749, -0.00375049, -0.03602223, ..., -0.03341193,
         -0.02006156,  0.01481214],
        [-0.03684749, -0.00375049, -0.03602223, ..., -0.03341193,
         -0.02006156,  0.01481214],
        ...,
        [-0.04051945, -0.06939801, -0.00165143, ...,  0.01269059,
          0.02405923, -0.00913538],
        [-0.04051945, -0.06939801, -0.00165143, ...,  0.01269059,
          0.02405923, -0.00913538],
        [-0.04051945, -0.06939801, -0.00165143, ...,  0.01269059,
          0.02405923, -0.00913538]]], dtype=float32)>

In [36]:
sample_embed_learned == sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=bool, numpy=
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])>

#Model 2 - Recurrent Neural Network (RNN)

In [37]:
# Create a simple LSTM model

inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x)
# x = tf.keras.layers.Dense(64, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model_2 = tf.keras.models.Model(inputs,outputs)
model_2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 177,473 (693.25 KB)

 Trainable params: 177,473 (693.25 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [39]:
model_2_history = model_2.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8007 - loss: 0.4435 - val_accuracy: 0.7756 - val_loss: 0.4787
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8321 - loss: 0.3789 - val_accuracy: 0.7808 - val_loss: 0.4900
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8333 - loss: 0.3719 - val_accuracy: 0.7808 - val_loss: 0.4953
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8392 - loss: 0.3643 - val_accuracy: 0.7690 - val_loss: 0.5001
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8547 - loss: 0.3446 - val_accuracy: 0.7664 - val_loss: 0.5055


In [40]:
model_2.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7626 - loss: 0.5208


[0.5055201053619385, 0.7664042115211487]

In [41]:
model_2_preds = model_2.predict(val_sentences)
print(model_2_preds[:5])
print(val_labels[:5])

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
[[0.559247  ]
 [0.6271052 ]
 [0.9447078 ]
 [0.10007577]
 [0.09563868]]
[0 0 1 1 1]


In [42]:
model_2_preds = tf.squeeze(tf.round(model_2_preds))

model_2_results = calculate_result(val_labels,model_2_preds)
model_2_results

{'accuracy': 0.7664041994750657,
 'precision': 0.7746244390911647,
 'recall': 0.7664041994750657,
 'f1': 0.7615262048991974}

In [43]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#Model 3 - GRU and RNN model

In [44]:
# Build an RNN using the GRU cell

inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(64, return_sequences = True)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model_3 = tf.keras.models.Model(inputs,outputs, name = 'model_3_GRU')
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 15, 64)              │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 165,313 (645.75 KB)

 Trainable params: 165,313 (645.75 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics  = ['accuracy'])

checkpoint3_cb = tf.keras.callbacks.ModelCheckpoint('model_3_checkpoints.weights.h5', save_weights_only = True)

model_3_history = model_3.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [checkpoint3_cb])

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.7924 - loss: 0.4489 - val_accuracy: 0.7717 - val_loss: 0.5241
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.8413 - loss: 0.3656 - val_accuracy: 0.7651 - val_loss: 0.4921
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8535 - loss: 0.3466 - val_accuracy: 0.7756 - val_loss: 0.5402
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8576 - loss: 0.3413 - val_accuracy: 0.7782 - val_loss: 0.5188
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8685 - loss: 0.3210 - val_accuracy: 0.7769 - val_loss: 0.5322


In [46]:
model_3.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7743 - loss: 0.5483


[0.5322494506835938, 0.7769029140472412]

In [47]:
model_3_preds_proba = model_3.predict(val_sentences)
print(model_3_preds_proba[:5])
print(val_labels[:5])

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
[[0.6124504 ]
 [0.85123307]
 [0.9809238 ]
 [0.04756345]
 [0.14112051]]
[0 0 1 1 1]


In [48]:
model_3_preds = tf.squeeze(tf.round(model_3_preds_proba))
model_3_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 0., 0.], dtype=float32)>

In [49]:
model_3_resulsts = calculate_result(val_labels,model_3_preds)
model_3_resulsts

{'accuracy': 0.7769028871391076,
 'precision': 0.7831567504504889,
 'recall': 0.7769028871391076,
 'f1': 0.7731247281588468}

#Model 4 - Bidirectional RNN

In [50]:
# Build a Bidirectional RNN

inputs = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model_4 = tf.keras.models.Model(inputs, outputs, name = 'Model_4_Bidirectional')
model_4.summary()

Model: "Model_4_Bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 226,945 (886.50 KB)

 Trainable params: 226,945 (886.50 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics  = ['accuracy'])

checkpoint4_cb = tf.keras.callbacks.ModelCheckpoint('model_4_checkpoints.weights.h5', save_weights_only = True)

model_4_history = model_4.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [checkpoint4_cb])

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.8196 - loss: 0.4074 - val_accuracy: 0.7730 - val_loss: 0.5689
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8800 - loss: 0.2933 - val_accuracy: 0.7507 - val_loss: 0.5648
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8894 - loss: 0.2751 - val_accuracy: 0.7677 - val_loss: 0.6078
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.8943 - loss: 0.2549 - val_accuracy: 0.7651 - val_loss: 0.5893
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9008 - loss: 0.2363 - val_accuracy: 0.7625 - val_loss: 0.7217


In [52]:
model_4.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7586 - loss: 0.7320


[0.7216880321502686, 0.7624672055244446]

In [53]:
model_4_preds_proba = model_4.predict(val_sentences)
print(model_4_preds_proba[:5])
print(val_labels[:5])

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
[[0.75352305]
 [0.4809597 ]
 [0.9874467 ]
 [0.03942124]
 [0.14974205]]
[0 0 1 1 1]


In [54]:
model_4_preds = tf.squeeze(tf.round(model_4_preds_proba))
model_4_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 0., 1., 0., 0.], dtype=float32)>

In [55]:
model_4_results = calculate_result(val_labels,model_4_preds)
model_4_results

{'accuracy': 0.7624671916010499,
 'precision': 0.7651076306104088,
 'recall': 0.7624671916010499,
 'f1': 0.7596743073939078}

#Model 5 - Convolutional Neural Networks for Text/Sequences

In [61]:
# Create 1D cnn layer to model sequences

inputs = tf.keras.layers.Input(shape = (1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(64, kernel_size = 5, strides = 1, padding = 'valid',activation = 'relu')(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model_5 = tf.keras.models.Model(inputs,outputs, model_name = 'Model_5_CNN')
model_5.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │         128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 11, 64)              │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_3           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 169,089 (660.50 KB)

 Trainable params: 169,089 (660.50 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
model_5.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics  = ['accuracy'])

checkpoint5_cb = tf.keras.callbacks.ModelCheckpoint('model_5_checkpoints.weights.h5', save_weights_only = True)

model_5_history = model_5.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [checkpoint5_cb])

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8063 - loss: 0.4309 - val_accuracy: 0.7677 - val_loss: 0.5533
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8508 - loss: 0.3364 - val_accuracy: 0.7756 - val_loss: 0.5447
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8656 - loss: 0.3099 - val_accuracy: 0.7756 - val_loss: 0.6036
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8750 - loss: 0.2927 - val_accuracy: 0.7677 - val_loss: 0.6118
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8822 - loss: 0.2765 - val_accuracy: 0.7664 - val_loss: 0.6630


In [63]:
model_5.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7540 - loss: 0.7040


[0.6629611253738403, 0.7664042115211487]

In [64]:
model_5_preds_prob = model_5.predict(val_sentences)
model_5_preds_prob[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


array([[0.88297945],
       [0.73151654],
       [0.98836464],
       [0.0235243 ],
       [0.19645943]], dtype=float32)

In [65]:
model_5_preds = tf.squeeze(tf.round(model_5_preds_prob))
model_5_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 0., 0.], dtype=float32)>

In [66]:
model_5_results = calculate_result(val_labels,model_5_preds)
model_5_results

{'accuracy': 0.7664041994750657,
 'precision': 0.7679094748917975,
 'recall': 0.7664041994750657,
 'f1': 0.7642857598265803}

#Model 6 - TensorFlow Hub Pretrained Sentence Encoder

In [68]:
import tensorflow_hub as hub

embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [72]:
sample_sequence = 'there is a flood in my street'

embed_sample = embed([sample_sequence,
                      'When you use universal sentence encoder on a sentence, it turns it into numbers'])
embed_sample[0].shape

TensorShape([512])

In [97]:
# Define a custom layer
class MyLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.sentence_encoder_layer = hub.KerasLayer(
            'https://tfhub.dev/google/universal-sentence-encoder/4',
            dtype=tf.string,
            trainable=False,
            name='USE'
        )

    def call(self, inputs):
        return self.sentence_encoder_layer(inputs)

In [98]:
inputs = tf.keras.layers.Input(shape=(), dtype = tf.string)
x = MyLayer()(inputs)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model_6 = tf.keras.models.Model(inputs,outputs, name = 'Model_6')
model_6.summary()

Model: "Model_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None)                      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ my_layer (MyLayer)                   │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,897 (128.50 KB)

 Trainable params: 32,897 (128.50 KB)

 Non-trainable params: 0 (0.00 B)

In [99]:
model_6.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics  = ['accuracy'])

checkpoint6_cb = tf.keras.callbacks.ModelCheckpoint('model_6_checkpoints.weights.h5', save_weights_only = True)

model_6_history = model_6.fit(train_sentences,train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [checkpoint6_cb])

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.7509 - loss: 0.5729 - val_accuracy: 0.7979 - val_loss: 0.4538
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8113 - loss: 0.4177 - val_accuracy: 0.8110 - val_loss: 0.4385
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8157 - loss: 0.4027 - val_accuracy: 0.8163 - val_loss: 0.4313
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8266 - loss: 0.3895 - val_accuracy: 0.8163 - val_loss: 0.4268
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.8306 - loss: 0.3841 - val_accuracy: 0.8176 - val_loss: 0.4257


In [101]:
model_6.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7968 - loss: 0.4755


[0.42566826939582825, 0.817585289478302]

In [106]:
model_6_preds_proba = model_6.predict(val_sentences)
model_6_preds_proba[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([[0.17381077],
       [0.79871917],
       [0.9854499 ],
       [0.21956293],
       [0.7412725 ]], dtype=float32)

In [110]:
model_6_preds = tf.squeeze(tf.round(model_6_preds_proba))
model_6_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 1.], dtype=float32)>

In [111]:
model_6_results = calculate_result(val_labels, model_6_preds)
model_6_results

{'accuracy': 0.8175853018372703,
 'precision': 0.818425972725479,
 'recall': 0.8175853018372703,
 'f1': 0.8166250648115712}